In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pathlib
data_root_orig = tf.keras.utils.get_file(origin='https://commondatastorage.googleapis.com/books1000/notMNIST_small.tar.gz',
                                         fname='notMNIST_small', untar=True)
data_root = pathlib.Path(data_root_orig)
print(data_root)

/home/maxim/.keras/datasets/notMNIST_small


In [2]:
for letter_dir in data_root.iterdir():
  print(letter_dir)

/home/maxim/.keras/datasets/notMNIST_small/F
/home/maxim/.keras/datasets/notMNIST_small/B
/home/maxim/.keras/datasets/notMNIST_small/H
/home/maxim/.keras/datasets/notMNIST_small/G
/home/maxim/.keras/datasets/notMNIST_small/D
/home/maxim/.keras/datasets/notMNIST_small/I
/home/maxim/.keras/datasets/notMNIST_small/C
/home/maxim/.keras/datasets/notMNIST_small/A
/home/maxim/.keras/datasets/notMNIST_small/J
/home/maxim/.keras/datasets/notMNIST_small/E


In [3]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count

18724

In [4]:
all_image_paths[:10]

import IPython.display as display

for n in range(5):
  image_path = random.choice(all_image_paths)
  display.display(display.Image(image_path))
  print(image_path)

/home/maxim/.keras/datasets/notMNIST_small/C/RHVzdFBpZWNlLm90Zg==.png


/home/maxim/.keras/datasets/notMNIST_small/F/SHVtYW5zdDUzMSBCVCBCb2xkLnR0Zg==.png


/home/maxim/.keras/datasets/notMNIST_small/I/MTIgV2FsYmF1bSBJdGFsaWMgMTMyNjMudHRm.png


/home/maxim/.keras/datasets/notMNIST_small/J/QXZlbmlyIDQ1IEJvb2sgT2JsaXF1ZS5wZmI=.png


/home/maxim/.keras/datasets/notMNIST_small/J/SG91c2VNb3ZlbWVudHMtU2lnbi5vdGY=.png


In [5]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

In [6]:
label_to_index = dict((name, index) for index, name in enumerate(label_names))
label_to_index

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9}

In [7]:
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:10])
print("First 10 paths: ", all_image_paths[:10])

First 10 labels indices:  [1, 0, 7, 0, 3, 1, 0, 9, 8, 6]
First 10 paths:  ['/home/maxim/.keras/datasets/notMNIST_small/B/RnV0dXJhQlQtRXh0cmFCbGFja0NvbmRJdGFsaWMub3Rm.png', '/home/maxim/.keras/datasets/notMNIST_small/A/Q2xhcmVuZG9uIEh2IEJUIEhlYXZ5LnR0Zg==.png', '/home/maxim/.keras/datasets/notMNIST_small/H/SGVhZGxpbmUgV2lkZSBCb2xkSXRhbGljLnR0Zg==.png', '/home/maxim/.keras/datasets/notMNIST_small/A/R290aGljIDcyMCBJdGFsaWMgQlQudHRm.png', '/home/maxim/.keras/datasets/notMNIST_small/D/SHViYnViIFNTaSBCb2xkLnR0Zg==.png', '/home/maxim/.keras/datasets/notMNIST_small/B/R2FydGhHcmFwaGljLUNvbmRlbnNlZC5vdGY=.png', '/home/maxim/.keras/datasets/notMNIST_small/A/R291ZHlDYXRhbG9ndWVNVFN0ZC1JdGFsaWMub3Rm.png', '/home/maxim/.keras/datasets/notMNIST_small/J/Q3VzaGluZ0l0Y1RFRUhlYS50dGY=.png', '/home/maxim/.keras/datasets/notMNIST_small/I/RXJhc0JRLUxpZ2h0Lm90Zg==.png', '/home/maxim/.keras/datasets/notMNIST_small/G/QXVndXN0LUxpZ2h0Lm90Zg==.png']


In [8]:
img_path = all_image_paths[0]
img_path
img_raw = tf.io.read_file(img_path)
print(repr(img_raw)[:100]+"...")

img_tensor = tf.image.decode_image(img_raw)

print(img_tensor.shape)
print(img_tensor.dtype)

<tf.Tensor: shape=(), dtype=string, numpy=b"\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\...
(28, 28, 1)
<dtype: 'uint8'>


In [9]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=1)
  image = tf.image.resize(image, [28, 28])
  return image

def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)

In [10]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
print(path_ds)

<TensorSliceDataset shapes: (), types: tf.string>


In [11]:
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))

for label in label_ds.take(10):
  print(label_names[label.numpy()])

print(label_ds)

image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
print(image_label_ds)

B
A
H
A
D
B
A
J
I
G
<TensorSliceDataset shapes: (), types: tf.int64>
<ZipDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>


In [24]:
ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# Кортежи распаковываются в позиционные аргументы отображаемой функции
def load_and_preprocess_from_path_label(path, label):
  return load_and_preprocess_image(path), label

image_label_ds = ds.map(load_and_preprocess_from_path_label)
image_label_ds


BATCH_SIZE = 32

# Установка размера буфера перемешивания, равного набору данных, гарантирует
# полное перемешивание данных.
# ds = image_label_ds.shuffle(buffer_size=image_count)
# ds = ds.repeat()
# ds = ds.batch(BATCH_SIZE)
# # `prefetch` позволяет датасету извлекать пакеты в фоновом режиме, во время обучения модели.
# ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

ds = image_label_ds.apply(
  tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
ds = ds.batch(BATCH_SIZE)
ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
ds

<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int32)>

In [25]:
from tensorflow import keras
model = keras.Sequential([
            keras.layers.Input(shape=(28, 28, 1)),
            tf.keras.layers.GlobalAveragePooling2D(),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dropout(0.4),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dropout(0.1),
            keras.layers.Dense(10, activation='sigmoid')
        ])
model.compile(optimizer='sgd',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
model.fit(ds, steps_per_epoch=10, epochs=10)

Train for 10 steps
Epoch 1/10
10/10 [==============================] - 2s 235ms/step - loss: 119.0451 - accuracy: 0.8781
Epoch 2/10
10/10 [==============================] - 0s 4ms/step - loss: 103.6883 - accuracy: 1.0000
Epoch 3/10
10/10 [==============================] - 0s 5ms/step - loss: 103.0410 - accuracy: 1.0000
Epoch 4/10
10/10 [==============================] - 0s 5ms/step - loss: 99.5868 - accuracy: 1.0000
Epoch 5/10
10/10 [==============================] - 0s 5ms/step - loss: 104.4078 - accuracy: 1.0000
Epoch 6/10
10/10 [==============================] - 0s 5ms/step - loss: 97.2842 - accuracy: 0.9969
Epoch 7/10
10/10 [==============================] - 0s 5ms/step - loss: 110.0924 - accuracy: 0.9969
Epoch 8/10
10/10 [==============================] - 0s 5ms/step - loss: 108.5813 - accuracy: 1.0000
Epoch 9/10
10/10 [==============================] - 0s 5ms/step - loss: 102.5408 - accuracy: 1.0000
Epoch 10/10
10/10 [==============================] - 0s 5ms/step - loss: 106.4946

In [ ]:
ds